# Predicting sentiment of smartphone reviews from Ukrainian eCommerce marketplace

# Build BERT model v2: oversample negative class to remove class imbalance in target variable

## Read clean_data and preprocess data

In [1]:
#Import necessary libraries
from datetime import datetime
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import dateparser
from datetime import timedelta
from datetime import date
import ktrain
from ktrain import text
import advertools as adv
from langdetect import detect
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

%config IPCompleter.greedy=True

#Remove limitations for displaying of tables in pandas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [ ]:
data_clean = pd.read_csv('smartphone_reviews_data_clean.csv')
print(data_clean.head())

In [66]:
pos_features = data_clean[data_clean['review_sentiment']=='1'][['full_text', 'review_sentiment']]
neg_features = data_clean[data_clean['review_sentiment']=='0'][['full_text', 'review_sentiment']]
neg_features = neg_features.sample(len(pos_features), replace=True)
len(neg_features)

2925

In [68]:
resampled_data = data_clean[data_clean['review_sentiment']=='1'][['full_text', 'review_sentiment']].append(neg_features)

In [69]:
resampled_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5850 entries, 8 to 1592
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   full_text         5850 non-null   object
 1   review_sentiment  5850 non-null   object
dtypes: object(2)
memory usage: 137.1+ KB


In [70]:
#Encode target variable (format required for the modeling)
target = pd.get_dummies(resampled_data.review_sentiment)
resampled_data['0'] = target.iloc[:, 0]
resampled_data['1'] = target.iloc[:, 1]

In [71]:
resampled_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5850 entries, 8 to 1592
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   full_text         5850 non-null   object
 1   review_sentiment  5850 non-null   object
 2   0                 5850 non-null   uint8 
 3   1                 5850 non-null   uint8 
dtypes: object(2), uint8(2)
memory usage: 148.5+ KB


In [ ]:
### Preprocessing: train/test split and preprocessing pipeline with Bert mode

For training the model we will use the column full_text which contains all text from reviews (review_text, product_advatages, product_disadvantages). 

In [72]:
#Split dataset into training and test. Test dataframe can be used later to check model performance on the unseen data.
#Add stratify parameter to ensure the same proportions of examples in each class as observed in the original dataset.
df_train, df_test = train_test_split(resampled_data, test_size=0.2, stratify=resampled_data.review_sentiment)

In [73]:
#Check proportions of examples in each class for training and test dataset.
display(round(df_test.groupby('review_sentiment').count()/df_test.shape[0], 2))
display(round(df_train.groupby('review_sentiment').count()/df_train.shape[0], 2))

,full_text,0,1
review_sentiment,,,
0,0.5,0.5,0.5
1,0.5,0.5,0.5


,full_text,0,1
review_sentiment,,,
0,0.5,0.5,0.5
1,0.5,0.5,0.5


In [74]:
#Data preprocessing with bert mode
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(df_train, 
                                                                   'full_text', # name of column containing review text
                                                                   label_columns=['0', '1'],
                                                                   maxlen=200, 
                                                                   max_features=100000,
                                                                   preprocess_mode='bert',
                                                                   val_pct=0.1)

preprocessing train...
language: ru


Is Multi-Label? False
preprocessing test...
language: ru


## Building Bert model with ktrain

In [75]:
#Initializing pre-trained Bert model
model = text.text_classifier(name='bert',
                             train_data=(x_train, y_train),
                             preproc=preproc)

Is Multi-Label? False
maxlen is 200
done.


In [76]:
#View loaded model architecture summary
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        [(None, 200)]        0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      [(None, 200)]        0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 200, 768), ( 91812096    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 200, 768)     1536        Input-Segment[0][0]              
_______________________________________________________________________________________

In [77]:
#Wrap the model and data in a Learner object
learner = ktrain.get_learner(model=model,
                             train_data=(x_train, y_train),
                             val_data=(x_test, y_test),
                             batch_size=6)

In [78]:
#Set learner to use default weight decay rate 
learner.get_weight_decay()
learner.set_weight_decay()

C:\Users\Anna\anaconda3\lib\site-packages\ktrain\core.py:357: UserWarning: recompiling model to use AdamWeightDecay as opimizer with weight decay of 0.01
  warnings.warn('recompiling model to use AdamWeightDecay as opimizer with weight decay of %s' % (wd) )


**Weight decay** 
Weight decay is a form of regularization, we will use it in order to reduce overfitting caused by target classes imbalance.
By default, ktrain uses no weight decay, but for our model we will set default weight decay rate of 0.01 implemented using the AdamWeightDecay optimizer.

In [79]:
#Create TensorBoard calback to use it later to analyse model performance
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir='/tmp/Tensor_Board', histogram_freq=0, write_graph=True, write_images=True)

#Training the model 
learner.fit_onecycle(lr=1e-5,
                     epochs=3,
                    callbacks=[tbCallBack])



begin training using onecycle policy with max lr of 1e-05...
Epoch 1/3
702/702 [==============================] - 6274s 9s/step - loss: 0.5982 - accuracy: 0.6641 - val_loss: 0.3351 - val_accuracy: 0.8568
Epoch 2/3
702/702 [==============================] - 20732s 30s/step - loss: 0.2969 - accuracy: 0.8811 - val_loss: 0.1728 - val_accuracy: 0.9316
Epoch 3/3
702/702 [==============================] - 5021s 7s/step - loss: 0.1090 - accuracy: 0.9639 - val_loss: 0.1255 - val_accuracy: 0.9551


In [107]:
#Save trained model
learner.save_model('sentiment_prediction_2_3epochs')
learner.model.save_weights("sentiment_prediction_v2")


# Step 4: Evaluating model performance

## Check model performance metrics on the Training dataset

In [50]:
#Load trained model
learner.model.load_weights("sentiment_prediction_v2")

In [112]:
#Analyse TensorBoard graphics
%reload_ext tensorboard
%tensorboard --logdir /tmp/Tensor_Board

Reusing TensorBoard on port 6006 (pid 22280), started 0:13:19 ago. (Use '!kill 22280' to kill it.)

In [109]:
#Check learner performance metrics
learner.validate()

              precision    recall  f1-score   support

           0       0.94      0.97      0.96       239
           1       0.97      0.93      0.95       229

    accuracy                           0.96       468
   macro avg       0.96      0.95      0.96       468
weighted avg       0.96      0.96      0.96       468



array([[233,   6],
       [ 15, 214]], dtype=int64)